In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from IPython.display import display
from scipy.stats import skew

# Load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.fillna(value=-999.0,inplace=True)
test.fillna(value=-999.0,inplace=True)


list_feature_Nan = []
for i in train.select_dtypes(exclude=['object']).columns:
    if (train[i] == -999.0).astype(int).sum() > 0:
        list_feature_Nan.append(i)


In [8]:
for i in list_feature_Nan:
    train[i].replace(-999.0,train[i].mean(),inplace=True)
    test[i].replace(-999.0,train[i].mean(),inplace=True)

y = np.array(pd.DataFrame(np.log(train.SalePrice)))
y2 = np.array(pd.DataFrame(np.log(train.SalePrice)))

mean_saleprice = pd.DataFrame(train.groupby(['GrLivArea'])['LotArea'].mean())
mean_saleprice.columns = ['LotArea_bis']

train.drop('SalePrice',axis=1,inplace=True)    

all_data = all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],test.loc[:,'MSSubClass':'SaleCondition']))
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna()))
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index


train[skewed_feats] = np.log1p(train[skewed_feats])
test[skewed_feats] = np.log1p(test[skewed_feats])

train[skewed_feats] = train[skewed_feats].fillna(all_data[skewed_feats].mean())
test[skewed_feats] = test[skewed_feats].fillna(all_data[skewed_feats].mean())

In [9]:
#Label Encoder
le = preprocessing.LabelEncoder()
train_str = train.select_dtypes(include=['object'])
test_str = test.select_dtypes(include=['object']) 
display(train_str.head())

train.drop(train_str.columns.values,axis=1,inplace=True)
test.drop(train_str.columns.values,axis=1,inplace=True)

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,-999,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,-999,-999,-999,WD,Normal
1,RL,Pave,-999,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,-999,-999,-999,WD,Normal
2,RL,Pave,-999,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,-999,-999,-999,WD,Normal
3,RL,Pave,-999,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,-999,-999,-999,WD,Abnorml
4,RL,Pave,-999,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,-999,-999,-999,WD,Normal


In [10]:
train_str_dum = pd.get_dummies(train_str)
test_str_dum = pd.get_dummies(test_str)

columns_dum = list(set(train_str_dum) & set(test_str_dum))

train_str_dum = train_str_dum[columns_dum]
test_str_dum = test_str_dum[columns_dum]

#New train and New test
train_flo = train.select_dtypes(exclude=['object'])
test_flo = test.select_dtypes(exclude=['object']) 

new_train = pd.merge(train_flo,train_str_dum,left_index=True,right_index=True)
new_test = pd.merge(test_flo,test_str_dum,left_index=True,right_index=True)

display(new_train.head())
print(new_train.columns.values)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,Neighborhood_MeadowV,Exterior2nd_HdBoard,Condition1_Norm,MasVnrType_None,BsmtQual_-999.0,ExterCond_Po,HouseStyle_2.5Unf,SaleType_ConLI,SaleType_ConLw,Neighborhood_ClearCr
0,1,4.110874,65.0,9.042040,7,5,2003,2003,5.283204,6.561031,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,3.044522,80.0,9.169623,6,8,1976,1976,0.000000,6.886532,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,4.110874,68.0,9.328212,7,5,2001,2002,5.093750,6.188264,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,4.262680,60.0,9.164401,7,5,1915,1970,0.000000,5.379897,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,4.110874,84.0,9.565284,8,5,2000,2000,5.860786,6.486161,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


['Id' 'MSSubClass' 'LotFrontage' 'LotArea' 'OverallQual' 'OverallCond'
 'YearBuilt' 'YearRemodAdd' 'MasVnrArea' 'BsmtFinSF1' 'BsmtFinSF2'
 'BsmtUnfSF' 'TotalBsmtSF' '1stFlrSF' '2ndFlrSF' 'LowQualFinSF' 'GrLivArea'
 'BsmtFullBath' 'BsmtHalfBath' 'FullBath' 'HalfBath' 'BedroomAbvGr'
 'KitchenAbvGr' 'TotRmsAbvGrd' 'Fireplaces' 'GarageYrBlt' 'GarageCars'
 'GarageArea' 'WoodDeckSF' 'OpenPorchSF' 'EnclosedPorch' '3SsnPorch'
 'ScreenPorch' 'PoolArea' 'MiscVal' 'MoSold' 'YrSold' 'Electrical_SBrkr'
 'BsmtFinType2_GLQ' 'GarageType_Basment' 'MSZoning_FV'
 'Neighborhood_IDOTRR' 'MSZoning_C (all)' 'LandContour_HLS'
 'Condition1_PosN' 'Condition1_RRNn' 'Exterior2nd_Brk Cmn'
 'BsmtCond_-999.0' 'ExterQual_Fa' 'BsmtQual_Fa' 'GarageFinish_RFn'
 'MiscFeature_Gar2' 'MiscFeature_Othr' 'FireplaceQu_Gd'
 'Neighborhood_BrDale' 'Neighborhood_Somerst' 'BsmtQual_Ex'
 'Exterior2nd_Stucco' 'BsmtFinType1_GLQ' 'Condition1_PosA'
 'Exterior2nd_Stone' 'SaleCondition_Partial' 'Condition1_RRNe'
 'GarageCond_Gd' 'Exterior

In [11]:
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA, KernelPCA, TruncatedSVD, SparsePCA

#train_clf = new_train.drop('SalePrice',axis=1)
train_clf = new_train.copy()
train_clf.drop('Id',axis=1,inplace=True)

index = pd.DataFrame(test.Id,columns = ['Id'])
test_clf = new_test.drop('Id',axis=1)

train_clf2 = train_clf.drop(['LotFrontage','MasVnrArea','GarageYrBlt'],axis=1)
train_clf2 = pd.merge(train_clf2,train[['LotFrontage','MasVnrArea','GarageYrBlt']],left_index=True,right_index=True)
train_clf2['tot_sf'] = train_clf2['TotalBsmtSF'] + train_clf2['GrLivArea']
train_clf2['ratio_fl'] = train_clf2['2ndFlrSF'] / train_clf2['1stFlrSF'] 
train_clf2['garage_ex'] = (train_clf2['GarageQual_Gd'] + train_clf2['GarageQual_TA'] + train_clf2['GarageQual_Fa'] + train_clf2['GarageQual_Po']) * (train_clf2['GarageCond_Ex'])

clus = KernelPCA(n_components = 25)
train_clf2_pca = clus.fit_transform(train_clf2)
train_clf3 = pd.merge(train_clf2,pd.DataFrame(train_clf2_pca),left_index=True,right_index=True)

test_clf2 = test_clf.drop(['LotFrontage','MasVnrArea','GarageYrBlt'],axis=1)
test_clf2 = pd.merge(test_clf2,test[['LotFrontage','MasVnrArea','GarageYrBlt']],left_index=True,right_index=True)
test_clf2['tot_sf'] = test_clf2['TotalBsmtSF'] + test_clf2['GrLivArea']
test_clf2['ratio_fl'] = test_clf2['2ndFlrSF']  / test_clf2['1stFlrSF']
test_clf2['garage_ex'] = (test_clf2['GarageQual_Gd'] + test_clf2['GarageQual_TA'] + test_clf2['GarageQual_Fa'] + test_clf2['GarageQual_Po']) * (test_clf2['GarageCond_Ex'])

test_clf3 = pd.merge(test_clf2,pd.DataFrame(clus.transform(test_clf2)),left_index=True,right_index=True)

index = pd.DataFrame(test.Id,columns = ['Id'])


def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def score(clf, train_np, random_state, folds, target, length):
    kf = KFold(n = length , n_folds=folds, shuffle = True, random_state = random_state)
    for itrain, itest in kf:
        Xtr, Xte = train_np[itrain], train_np[itest]
        ytr, yte = target[itrain], target[itest]
        clf.fit(Xtr, ytr.ravel())
        pred = pd.DataFrame(clf.predict(Xte))
        return rmse(yte, pred)
    return rmse(y, pred)

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
from scipy import stats

train_clf4 = train_clf3.copy()
test_clf4 = test_clf3.copy()


In [13]:
for i in train_clf4.select_dtypes(include=['float64']).columns:
    if train_clf4[i].min() > test_clf4[i].min():
        train_clf4[i][train_clf4[i]==train_clf4[i].min()] = test_clf4[i].min()

for i in train_clf4.select_dtypes(include=['float64']).columns:
    if train_clf4[i].min() < test_clf4[i].min():
        test_clf4[i][test_clf4[i]==test_clf4[i].min()] = train_clf4[i].min()     

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
for i in train_clf4.select_dtypes(include=['float64']).columns:
    if train_clf4[i].max() > test_clf4[i].max():
        test_clf4[i][test_clf4[i]==test_clf4[i].max()] = train_clf4[i].max()

for i in train_clf4.select_dtypes(include=['float64']).columns:
    if train_clf4[i].max() < test_clf4[i].max():
        train_clf4[i][train_clf4[i]==train_clf4[i].max()] = test_clf4[i].max()  

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
import xgboost as xgb
from sklearn.ensemble.gradient_boosting import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, RandomTreesEmbedding
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.cluster import KMeans
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import Ridge, LassoCV
from sklearn.preprocessing import RobustScaler

def frange(start, stop, step):
    i = start
    a = []
    while i < stop:
        yield i
        a.append(i)
        i += step
    return a

Cs1 = list(range(2,10,1))
#Cs1 = [10,20]
#Cs1 = [1000,2000,3000,4000,5000,6000]
res = []
res2 = []
res3 = []
res4 = []

train_np = np.array(train_clf3)
train_np2 = np.array(train_clf4)
robust_scaler = RobustScaler()

for C in Cs1:
    res3.append(score(LassoCV(alphas = [1, 0.1, 0.001, 0.0005], random_state = 0),train_np,random_state = 0, folds = C, target = y , length = 1450))
    res4.append(score(Ridge(alpha = 21, random_state = 0),train_np,random_state = 0, folds = C, target = y , length = 1450))
p3, = plt.plot(Cs1, res3,'g-o',label="V3")
p4, = plt.plot(Cs1, res4,'y-o',label="V4")
plt.legend([p3,p4])
plt.show()

/usr/local/lib/python3.4/dist-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of

In [16]:
index = pd.DataFrame(test.Id,columns = ['Id'])

In [17]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso

#clf = LassoCV(alphas = [ 1, 0.1, 0.001, 0.0005], max_iter = 1000)
clf = xgb.XGBRegressor(n_estimators = 6000, seed = 0, learning_rate = 0.01, max_depth = 3, subsample = 0.8, colsample_bytree = 0.8, colsample_bylevel = 0.8 )
clf2 = Ridge(alpha=21)
clf3 = LassoCV(alphas = [1, 0.1, 0.001, 0.0005])
Select = SelectKBest(f_regression, k = 268)

train_fin = np.array(Select.fit_transform(train_clf4,y))
test_fin = np.array(Select.transform(test_clf4))

train_fin_ridge = np.array(robust_scaler.fit_transform(train_clf3))
test_fin_ridge = np.array(robust_scaler.transform(test_clf3))

kf1 = KFold(n = 1450 , n_folds=7, random_state=0, shuffle = True)
#kf2 = KFold(n = 1450 , n_folds=39, random_state=0, shuffle = True)

i = 0
res1 = []
res2 = []

for itrain, itest in kf1:
    i = i + 1
    Xtr, Xte = train_fin[itrain], train_fin[itest]
    ytr, yte = y[itrain], y[itest]
    clf.fit(Xtr, ytr.ravel())
    if i == 1:
        pred1 = pd.DataFrame(clf.predict(test_fin))
        print("Fold 1 :", rmse(yte, pd.DataFrame(clf.predict(Xte))))
        res1.append(rmse(yte, pd.DataFrame(clf.predict(Xte))))
    if i > 1 :
        pred1 = pred1 + pd.DataFrame(clf.predict(test_fin))
        print("Fold ",i, " :", rmse(yte, pd.DataFrame(clf.predict(Xte))))
        res1.append(rmse(yte, pd.DataFrame(clf.predict(Xte))))

clf2.fit(train_fin_ridge, y)
pred2 = pd.DataFrame(clf2.predict(test_fin_ridge))
        
clf3.fit(train_fin_ridge, y)
pred3 = pd.DataFrame(clf3.predict(test_fin_ridge))
        
pred_1 = pred1/7 

print(pred2[np.exp(pred2).values<1].index.values)

for i in pred2[np.exp(pred2).values<1].index.values:
    pred2[pred2.index == i] = pred_1[pred_1.index == i]

print(pred2[pred2.values>12].index.values)

for i in pred2[pred2.values>12].index.values:
    pred2[pred2.index == i] = pred_1[pred_1.index == i]
    
print(np.mean(res1))
print(np.exp(pred2).head())

pred_1 = (pred_1+pred2+pred3)/3
pred_xg = pred_1

pred_1.columns = ['SalePrice']
pred_xg.columns = ['SalePrice']
pred3.columns = ['SalePrice']

pred_final_1 = pd.DataFrame(np.exp(pred_1), index = new_test.index, columns = ['SalePrice'])
pred_Lasso = pd.DataFrame(np.exp(pred3), index = new_test.index, columns = ['SalePrice'])
pred_xg = pd.DataFrame(np.exp(pred_xg), index = new_test.index, columns = ['SalePrice'])

pred_final = pred_final_1

pred_submit = pd.merge(index,pred_final,left_index=True,right_index=True)
pred_Lasso_submit = pd.merge(index,pred_Lasso,left_index=True,right_index=True)
pred_xg_submit = pd.merge(index,pred_xg,left_index=True,right_index=True)
pred_submit.head()

pred_submit.to_csv('XG_Ridge.csv',index=False)
pred_Lasso_submit.to_csv('Lasso.csv',index=False)
pred_xg_submit.to_csv('XGB.csv',index=False)

/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fold 1 : 0    0.100052
dtype: float64
Fold  2  : 0    0.132465
dtype: float64
Fold  3  : 0    0.132695
dtype: float64
Fold  4  : 0    0.143382
dtype: float64
Fold  5  : 0    0.108891
dtype: float64
Fold  6  : 0    0.095202
dtype: float64
Fold  7  : 0    0.104332
dtype: float64


/usr/local/lib/python3.4/dist-packages/sklearn/linear_model/coordinate_descent.py:1077: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.4/dist-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[ 728 1116]
[   2    3    4    5    6    8   10   15   16   17   18   19   20   21   22
   23   24   25   26   27   28   29   30   32   33   34   35   36   37   38
   40   42   43   44   45   46   47   51   54   56   58   61   69   78   82
  105  107  112  114  115  116  119  122  123  135  136  137  138  139  143
  144  145  146  147  148  151  152  162  163  165  166  167  168  169  170
  171  172  173  174  176  177  178  179  180  181  182  183  184  200  201
  202  203  204  205  206  207  208  209  210  211  212  213  214  215  216
  217  218  219  220  221  222  223  224  225  226  227  228  229  230  231
  232  233  234  235  236  237  238  239  240  241  242  243  244  245  246
  247  248  249  250  251  252  253  254  255  256  258  259  263  264  265
  267  268  276  277  278  279  280  281  282  283  284  285  286  287  290
  293  294  298  299  301  302  304  305  306  308  319  326  330  386  393
  395  401  402  403  404  405  406  407  408  409  410  411  412  414  415
